In [65]:
import datetime

apalis = {
    "alpha": ["/home/tjw/Downloads/apalis/GPS-alpha01.csv", "/home/tjw/Downloads/apalis/GPS-alpha02sorted.csv", "/home/tjw/Downloads/apalis/GPS-zeta01sorted.csv", "/home/tjw/Downloads/apalis/GPS-zeta02sorted.csv", "/home/tjw/Downloads/apalis/GPS-zeta03sorted.csv"],
    "beta": ["/home/tjw/Downloads/apalis/GPS-beta01sorted.csv", "/home/tjw/Downloads/apalis/GPS-eta02sorted.csv"],
    "gamma": ["/home/tjw/Downloads/apalis/GPS-gamma01.csv"],
    "delta": ["/home/tjw/Downloads/apalis/GPS-delta01sorted.csv"],
    #"epsilon": [],
    #"zeta": [],
    #"eta": []
}
apalis_output = "/home/tjw/Downloads/apalis/sessions.json"

gopro = {
    "alpha": ["/home/tjw/Documents/gopro/alpha_json", "/home/tjw/Documents/gopro/zeta_json"],
    "beta": ["/home/tjw/Documents/gopro/beta_json", "/home/tjw/Documents/gopro/eta_json"],
    "gamma": ["/home/tjw/Documents/gopro/gamma_json", "/home/tjw/Documents/gopro/epsilon_json"],
    "delta": ["/home/tjw/Documents/gopro/delta_json"],
    #"epsilon": [],
    #"zeta": [],
    #"eta": []
}

canbus = {
    "apalis1": "/home/tjw/Documents/apalis23/apalis1-DEL_ETA.sql",
    "apalis2g": "/home/tjw/Documents/apalis23/apalis2-GAMMA.sql",
    "apalis2z1": "/home/tjw/Documents/apalis23/apalis2-ZETA1.sql",
    "apalis2z3": "/home/tjw/Documents/apalis23/apalis2-ZETA3.sql",
    "apalis3a": "/home/tjw/Documents/apalis23/apalis3-ALPHA.sql",
   #"apalis3e": "/home/tjw/Documents/apalis23/apalis3-ETA.sql",
    "apalis4a": "/home/tjw/Documents/apalis23/apalis4-ALPHA.sql",
   #"apalis4e": "/home/tjw/Documents/apalis23/apalis4-EPSILON.sql",
   #"apalis5b": "/home/tjw/Documents/apalis23/apalis5-BETA.sql",
    "apalis5z": "/home/tjw/Documents/apalis23/apalis5-ZETA.sql"
}

isoblue = {
    "alpha": { # merged with zeta
        "apalis3a": {"start": datetime.datetime(2023, 5,19),
                     "end":   datetime.datetime(2023, 5,24)},
        "apalis4a": {"start": datetime.datetime(2023, 5,25),
                     "end":   datetime.datetime(2023, 6,16)},
        "apalis2z1":{"start": datetime.datetime(2023, 8,19),
                     "end":   datetime.datetime(2023, 8,30)},
        "apalis5z": {"start": datetime.datetime(2023, 8,30),
                     "end":   datetime.datetime(2023, 9,15)},
        "apalis2z3":{"start": datetime.datetime(2023, 9,15),
                     "end":   datetime.datetime(2023,10,21)}
    },

    "beta": { # merged with eta
        # "apalis5b": {"start": datetime.datetime(2023, 5,25),
        #              "end":   datetime.datetime(2023, 6,23)},
        # "apalis3e": {"start": datetime.datetime(2023, 8,19),
        #              "end":   datetime.datetime(2023, 9,23)},
        "apalis1":  {"start": datetime.datetime(2023, 9,23),
                     "end":   datetime.datetime(2023,10,21)}
    },

    "gamma": { # merged with epsilon
        "apalis2g": {"start": datetime.datetime(2023, 5,25),
                     "end":   datetime.datetime(2023, 6,24)},
        # "apalis4e": {"start": datetime.datetime(2023, 8, 1),
        #              "end":   datetime.datetime(2023,10,21)}
    },

    "delta": { # not merged with any others
        "apalis1": {"start": datetime.datetime(2023, 5,25),
                    "end":   datetime.datetime(2023, 6,16)}
    }
}


maintenance = [ # all the dates of data swaps
    datetime.datetime(2023, 5, 9),
    datetime.datetime(2023, 5,19),
    datetime.datetime(2023, 5,25),
    datetime.datetime(2023, 6,16),
    datetime.datetime(2023, 6,23),
    datetime.datetime(2023, 8,19),
    datetime.datetime(2023, 8,30),
    datetime.datetime(2023, 9,15),
    datetime.datetime(2023, 9,23),
    datetime.datetime(2023,10,20)
]

In [66]:
# Temporary cell for sorting csv by time column
# import pandas as pd
# sheet = pd.read_csv("/home/tjw/Downloads/apalis/GPS-eta02.csv")
# sheet = sheet.sort_values(by="time")
# sheet.to_csv("/home/tjw/Downloads/apalis/GPS-eta02sorted.csv", index=False)

In [67]:
# Read Isoblue GPS data
from dateutil import parser
ZERO = datetime.timedelta(0)

a_data = {}

for tractor in apalis:
    a_data[tractor] = {"datetime":[], "lat":[], "long":[], "deltas": {}}
    for csv in apalis[tractor]:
        print(f"Parsing {csv} ...")
        with open(csv) as file:
            first = True
            for line in file.readlines()[1:]:
                # Parse data from CSV
                time, lat, lng = line.split(",")
                lat = float(lat)
                lng = float(lng.rstrip())
                time = parser.isoparse(time)
                
                # Add data to dict
                a_data[tractor]["datetime"].append(time)
                a_data[tractor]["lat"].append(lat)
                a_data[tractor]["long"].append(lng)
                
                # Calculate deltas
                if first:
                    otime = time
                    first = False
                else:
                    dtime = time - otime
                    if dtime < ZERO:
                        print(f"[NEGATIVE DELTA!]: {dtime} found at '{line}'")
                    otime = time
                    a_data[tractor]["deltas"][time] = dtime.total_seconds()


Parsing /home/tjw/Downloads/apalis/GPS-alpha01.csv ...
Parsing /home/tjw/Downloads/apalis/GPS-alpha02sorted.csv ...
Parsing /home/tjw/Downloads/apalis/GPS-zeta01sorted.csv ...
Parsing /home/tjw/Downloads/apalis/GPS-zeta02sorted.csv ...
Parsing /home/tjw/Downloads/apalis/GPS-zeta03sorted.csv ...
Parsing /home/tjw/Downloads/apalis/GPS-beta01sorted.csv ...
Parsing /home/tjw/Downloads/apalis/GPS-eta02sorted.csv ...
Parsing /home/tjw/Downloads/apalis/GPS-gamma01.csv ...
Parsing /home/tjw/Downloads/apalis/GPS-delta01sorted.csv ...


In [138]:
list(a_data.keys()) # a list of all the tractors

['alpha', 'beta', 'gamma', 'delta']

In [176]:
# Get sessions - will not stitch together
MINS = 5
threshold = 60*MINS # if more than MINS without updates, call it a new session
a_sessions = {}

for tractor in a_data:
    a_sessions[tractor] = []
    newTrack = True
    
    for i in range(len(a_data[tractor]["datetime"])):
        time = a_data[tractor]["datetime"][i]
        
        if newTrack:
            start = i
            newTrack = False
        if time in a_data[tractor]["deltas"]:
            if a_data[tractor]["deltas"][time] > threshold:
                if i - start > threshold/MINS: # ignore really short segments
                    a_sessions[tractor].append((start,i-1))
                newTrack = True
        else: # means we skipped to a new file
            if (i-1) - start > threshold/MINS: # ignore really short segments
                a_sessions[tractor].append((start,i-1))
            newTrack = True

In [177]:
print(f"a_sessions['alpha']: \n\t{',\n\t'.join([str(x) for x in a_sessions['alpha'][:5]])}...")

a_sessions['alpha']: 
	(1, 81620),
	(81622, 156056),
	(156058, 284511),
	(284513, 704899),
	(704901, 714262)...


In [178]:
# Get tracks - will stitch together post-smoothing (with splines?)
threshold = 60 # how many seconds allowed before breaking into a new track

for tractor in a_data:
    for s in range(len(a_sessions[tractor])):
        ss, se = a_sessions[tractor][s]
        tracks = []
        newTrack = True
        
        for i in range(ss, se+1):
            time = a_data[tractor]["datetime"][i]
            
            if newTrack:
                start = i
                newTrack = False
            if time in a_data[tractor]["deltas"]:
                if a_data[tractor]["deltas"][time] > threshold:
                    if i - start > threshold/MINS: # ignore truly tiny segments
                        tracks.append((start, i-1))
                    newTrack = True
            else:
                if (i-1) - start > threshold/MINS: # ignore truly tiny segments
                    tracks.append((start, i-1))
                newTrack = True
        
        # Don't forget final track
        if not newTrack:
            if (i-1) - start > threshold/MINS: # ignore truly tiny segments
                tracks.append((start, i-1))
        
        if len(tracks) > 1:
            a_sessions[tractor][s] = tracks
        else:
            a_sessions[tractor][s] = [a_sessions[tractor][s]]


In [180]:
print(f"a_sessions['alpha']: \n\t{',\n\t'.join([str(x) for x in a_sessions['alpha'][:5]])}...")


a_sessions['alpha']: 
	[(1, 22053), (22055, 31018), (31020, 81619)],
	[(81622, 91301), (91303, 156055)],
	[(156058, 169712), (169714, 172129), (172131, 173305), (173307, 174549), (174551, 263103), (263105, 284510)],
	[(284513, 338101), (338103, 589191), (589193, 595056), (595058, 599733), (599735, 610403), (610405, 652295), (652297, 664167), (664169, 668831), (668833, 671092), (671094, 704898)],
	[(704901, 713450), (713452, 714158), (714160, 714261)]...


In [181]:
# Get subtracks - will stitch together pre-smoothing w/ simple linear interp
for tractor in a_sessions:
    for s in range(len(a_sessions[tractor])):
        for b in range(len(a_sessions[tractor][s])):
            bs, be = a_sessions[tractor][s][b]
            subtracks = []
            
            otime = a_data[tractor]["datetime"][bs]
            startx = bs
            
            # Subtracks are 100% continuous sets of coordinates
            for i in range(bs, be+1):
                time = a_data[tractor]["datetime"][i]
                if (time - otime) > datetime.timedelta(seconds=1.4):
                    subtracks.append((startx, i-1))
                    startx = i
                otime = a_data[tractor]["datetime"][i]
            
            # Don't forget to check final subtrack
            if startx != i:
                subtracks.append((startx, i))
            
            if len(subtracks) > 1:
                a_sessions[tractor][s][b] = subtracks
            else:
                a_sessions[tractor][s][b] = [a_sessions[tractor][s][b]]

In [158]:
print(f"a_subtracks['alpha'][0]: \n\t{',\n\t'.join([str(x) for x in a_sessions['alpha'][0][0][:5]])}...")


a_subtracks['alpha'][0]: 
	(1, 324),
	(325, 441),
	(442, 561),
	(562, 682),
	(683, 683)...


In [182]:
from statistics import mean

for tractor in a_sessions:
    # Summary stats for sessions
    print(f"{tractor}:")
    print(f" - has {len(a_sessions[tractor])} segments recorded")
#     print(f" - tracks last " +
#           f"{(mean([end - start for start,end in a_sessions[tractor]])/60)/60:0.1f} " +
#           f"hours on avg")
    
    # Summary stats for tracks
    trackl = [len(subtracks) for subtracks in a_sessions[tractor]]
    print(f" - sessions have avg of {mean(trackl):0.2f} tracks")
    print(f" - most broken session has {max(trackl)} tracks")

alpha:
 - has 39 segments recorded
 - sessions have avg of 3.38 tracks
 - most broken session has 21 tracks
beta:
 - has 77 segments recorded
 - sessions have avg of 1.14 tracks
 - most broken session has 4 tracks
gamma:
 - has 7 segments recorded
 - sessions have avg of 1.29 tracks
 - most broken session has 3 tracks
delta:
 - has 3 segments recorded
 - sessions have avg of 1.00 tracks
 - most broken session has 1 tracks


In [160]:
import json

with open(apalis_output, mode="w") as file:
    json.dump(a_sessions, file)

In [161]:
import json

with open(apalis_output, mode="r") as file:
    a_sessions = json.load(file)

In [162]:
a_sessions["alpha"][0][0]
# ["tractor"]
#   [session]
#     [track]
#       [subtrack]

[[1, 324],
 [325, 441],
 [442, 561],
 [562, 682],
 [683, 683],
 [684, 797],
 [798, 917],
 [918, 1034],
 [1035, 1151],
 [1152, 1272],
 [1273, 1391],
 [1392, 1511],
 [1512, 1627],
 [1628, 1745],
 [1746, 1865],
 [1866, 1980],
 [1981, 2100],
 [2101, 2222],
 [2223, 2342],
 [2343, 2462],
 [2463, 2582],
 [2583, 2702],
 [2703, 2823],
 [2824, 2940],
 [2941, 3061],
 [3062, 3177],
 [3178, 3295],
 [3296, 3415],
 [3416, 3535],
 [3536, 3655],
 [3656, 3775],
 [3776, 3895],
 [3896, 4015],
 [4016, 4135],
 [4136, 4252],
 [4253, 4372],
 [4373, 4489],
 [4490, 4609],
 [4610, 4726],
 [4727, 4846],
 [4847, 4966],
 [4967, 5086],
 [5087, 5202],
 [5203, 5322],
 [5323, 5442],
 [5443, 5562],
 [5563, 5683],
 [5684, 5801],
 [5802, 5919],
 [5920, 6035],
 [6036, 6155],
 [6156, 6275],
 [6276, 6395],
 [6396, 6514],
 [6515, 6634],
 [6635, 6754],
 [6755, 6871],
 [6872, 6991],
 [6992, 7112],
 [7113, 7230],
 [7231, 7351],
 [7352, 7471],
 [7472, 7591],
 [7592, 7711],
 [7712, 7825],
 [7826, 7945],
 [7946, 8065],
 [8066, 8185

In [163]:
def r2s(input):
    return datetime.datetime.fromtimestamp(round(input.timestamp()))

In [164]:
def get_idx(df, item):
    return df.eq(item).idxmax()

In [183]:
# Create dataframes for each track
import pandas as pd
import numpy as np

a_frames = {}

for tractor in a_sessions:
    a_frames[tractor] = []
    
    for session in range(len(a_sessions[tractor])):
        ss = r2s(a_data[tractor]["datetime"][a_sessions[tractor][session][0][0][0]])
        se = r2s(a_data[tractor]["datetime"][a_sessions[tractor][session][-1][-1][-1]])
        
        times = np.arange(ss,se,datetime.timedelta(seconds=1))
        df = pd.DataFrame(data={
            "time": times,
            "lat": np.full_like(times, np.nan, dtype=float),
            "lon": np.full_like(times, np.nan, dtype=float)
        })
        
        # Insert GPS from each subtrack (they MUST be continuous in a_data!)
        for track in range(len(a_sessions[tractor][session])):
            print(f"Saving track {session}:{track} for tractor {tractor}")
            for bs,be in a_sessions[tractor][session][track]:
                i = get_idx(df["time"], r2s(a_data[tractor]["datetime"][bs]))
                j = get_idx(df["time"], r2s(a_data[tractor]["datetime"][be]))
                
                if (j-i) == (be-bs):
                    df.loc[i:j, "lat"] = a_data[tractor]["lat"][bs:be+1]
                    df.loc[i:j, "lon"] = a_data[tractor]["long"][bs:be+1]
                else:
                    print(f"**Subtrack ({bs},{be}) is end-of-line! Inserting data[{bs}:{be}] into dataframe[{i}:]")
                    df.loc[i:, "lat"] = a_data[tractor]["lat"][bs:be]
                    df.loc[i:, "lon"] = a_data[tractor]["long"][bs:be]
        
        a_frames[tractor].append(df.copy())

a_frames


Saving track 0:0 for tractor alpha
Saving track 0:1 for tractor alpha
Saving track 0:2 for tractor alpha
**Subtrack (81500,81619) is end-of-line! Inserting data[81500:81619] into dataframe[88747:]
Saving track 1:0 for tractor alpha
Saving track 1:1 for tractor alpha
**Subtrack (155938,156055) is end-of-line! Inserting data[155938:156055] into dataframe[81307:]
Saving track 2:0 for tractor alpha
Saving track 2:1 for tractor alpha
Saving track 2:2 for tractor alpha
Saving track 2:3 for tractor alpha
Saving track 2:4 for tractor alpha
Saving track 2:5 for tractor alpha
**Subtrack (284274,284510) is end-of-line! Inserting data[284274:284510] into dataframe[134356:]
Saving track 3:0 for tractor alpha
Saving track 3:1 for tractor alpha
Saving track 3:2 for tractor alpha
Saving track 3:3 for tractor alpha
Saving track 3:4 for tractor alpha
Saving track 3:5 for tractor alpha
Saving track 3:6 for tractor alpha
Saving track 3:7 for tractor alpha
Saving track 3:8 for tractor alpha
Saving track 3:

ValueError: Must have equal len keys and value when setting with an iterable

In [175]:
len(a_sessions["alpha"][31])

1